In [1]:
# pip install shiba shiba-model evaluate datasets wandb arabert  accelerate -U nltk torchmetrics==0.3.2

In [2]:
from typing import Dict
import pandas as pd
import numpy as np
import torchmetrics
import torch
import transformers
from datasets import load_dataset, Dataset
from transformers import HfArgumentParser, Trainer, EvalPrediction

from shiba import ShibaForClassification, CodepointTokenizer
from training.helpers import DataArguments, get_base_shiba_state_dict,get_model_hyperparams, ShibaClassificationArgs, \
    ClassificationDataCollator

<h1> Choose the model

In [3]:
model_path = '../checkpoint-611960.pt'
seg_enable = True
bert_model_name = "aubmindlab/bert-base-arabertv02"
apply_farasa=False
file_save = 'Submit_64_05'

# Pre-Process data ( if needed)

<h1> Read the files

In [4]:
transformers.logging.set_verbosity_info()
device = "cuda"
parser = HfArgumentParser((ShibaClassificationArgs, DataArguments))

prediction_label = 'label'

df_train = pd.read_csv("data/MADAR-Corpus-26-train.tsv",names=['Text', 'label'], sep="\t")
df_dev = pd.read_csv("data/MADAR-Corpus-26-dev.tsv",names=['Text', 'label'], sep="\t")
df_testOrignal = pd.read_csv("data/MADAR-Corpus-26-test.tsv",names=['Text', 'label'], sep="\t")

categories = {idx: cat_name for idx, cat_name in enumerate(set(df_train[prediction_label]))}
id_by_category = {val: key for key, val in categories.items()}

print("categories : ", categories)
print("id_by_category : ", id_by_category)
df_train = Dataset.from_pandas(df_train)
df_dev = Dataset.from_pandas(df_dev)
df_test = Dataset.from_pandas(df_testOrignal)

categories :  {0: 'TUN', 1: 'BAG', 2: 'ALG', 3: 'DAM', 4: 'TRI', 5: 'ASW', 6: 'RAB', 7: 'SAN', 8: 'SFX', 9: 'BAS', 10: 'MOS', 11: 'JER', 12: 'MUS', 13: 'CAI', 14: 'RIY', 15: 'KHA', 16: 'BEN', 17: 'FES', 18: 'DOH', 19: 'SAL', 20: 'AMM', 21: 'ALE', 22: 'MSA', 23: 'ALX', 24: 'JED', 25: 'BEI'}
id_by_category :  {'TUN': 0, 'BAG': 1, 'ALG': 2, 'DAM': 3, 'TRI': 4, 'ASW': 5, 'RAB': 6, 'SAN': 7, 'SFX': 8, 'BAS': 9, 'MOS': 10, 'JER': 11, 'MUS': 12, 'CAI': 13, 'RIY': 14, 'KHA': 15, 'BEN': 16, 'FES': 17, 'DOH': 18, 'SAL': 19, 'AMM': 20, 'ALE': 21, 'MSA': 22, 'ALX': 23, 'JED': 24, 'BEI': 25}


In [5]:
if seg_enable:
    from arabert.preprocess import ArabertPreprocessor

    arabert_prep = ArabertPreprocessor(model_name=bert_model_name,apply_farasa_segmentation=apply_farasa)
    # arabert_prep.preprocess()
    df_train = pd.DataFrame(df_train)
    df_test =  pd.DataFrame(df_test)
    df_dev = pd.DataFrame(df_dev)
    df_train['Text'] =  df_train['Text'].apply(arabert_prep.preprocess)
    df_test['Text'] =  df_test['Text'].apply(arabert_prep.preprocess)
    df_dev['Text']=df_dev['Text'].apply(arabert_prep.preprocess)
    df_train = Dataset.from_pandas(df_train)
    df_test = Dataset.from_pandas(df_test)
    df_dev = Dataset.from_pandas(df_dev)

<h1> Check files count

In [6]:
len(df_train), len(df_dev), len(df_test)

(41600, 5200, 5200)

<h1> Load the model

In [7]:
tokenizer = CodepointTokenizer()
model_hyperparams = {'dropout': 0.1, 'deep_transformer_stack_layers': 12, 'local_attention_window': 128}
print(model_hyperparams)
model = ShibaForClassification(vocab_size=len(categories), **model_hyperparams)
data_collator = ClassificationDataCollator()
print('Loading and using base shiba states from', model_path)
checkpoint_state_dict = torch.load(model_path)
model.shiba_model.load_state_dict(get_base_shiba_state_dict(checkpoint_state_dict))

{'dropout': 0.1, 'deep_transformer_stack_layers': 12, 'local_attention_window': 128}


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Loading and using base shiba states from ../checkpoint-611960.pt


<All keys matched successfully>

<h1>Input IDs Method

In [8]:
def process_example(example: Dict) -> Dict:
    return {
        'input_ids': tokenizer.encode(example['Text'])['input_ids'][:model.config.max_length],
        'labels': id_by_category[example[prediction_label]]
    }
def process_exampleTemp(example: Dict) -> Dict:
    return {
        'input_ids': tokenizer.encode(example['Text'])['input_ids'][:model.config.max_length],
        'labels': 0
    }

<h1> Compute Metrics

In [9]:
def compute_metrics(pred: EvalPrediction) -> Dict:
    try:
        # Convert predictions and labels to PyTorch tensors
        # label_probs = torch.tensor(pred.predictions)
        label_probs, embeddings = pred.predictions
        labels = torch.tensor(pred.label_ids)
        label_probs = torch.exp(torch.tensor(label_probs))  # undo the log in log softmax, get indices
        # # Compute accuracy
        # accuracy = torchmetrics.functional.accuracy(label_probs, labels, num_classes=len(categories))

        # Compute F1 score
        f1_score = torchmetrics.functional.f1(label_probs, labels, average='macro', num_classes=len(categories))

        # Compute recall
        recall = torchmetrics.functional.recall(label_probs, labels, average='macro', num_classes=len(categories))

        # Compute precision
        precision = torchmetrics.functional.precision(label_probs, labels, average='macro', num_classes=len(categories))

        # print("label_probs : ", label_probs, " labels : ", labels)

        metrics = {
            # 'accuracy': accuracy.item(),
            'f1_score': f1_score.item(),
            'recall': recall.item(),
            'precision': precision.item()
        }

        # print("metrics : ", metrics)
        # raise NotImplementedError

        return metrics
    except:
        print("pred : ", pred)
        print("pred.predictions : ", pred.predictions)
        print("label_probs : ", label_probs)
        print("label_probs.size : ", label_probs.size())
        print("labels : ", labels)
        print("labels.size() : ", labels.size())

        raise NotImplementedError

<h1> Fine-tune args

In [10]:
training_args = ShibaClassificationArgs(
    per_device_eval_batch_size=64,
    per_device_train_batch_size=64,
    data_seed=42,
    seed=42,
    do_eval=True,
    do_predict=True,
    do_train=True,
    dropout=0.5,
    eval_accumulation_steps=None,
    eval_delay=0,
    eval_steps=100,
    evaluation_strategy='steps',
    gradient_accumulation_steps=1,
    num_train_epochs=10,
    output_dir="fine_result",
    prediction_loss_only=False,
    report_to=[],
    run_name="fine_result",
    save_strategy='no',
)

PyTorch: setting up devices


<h1> Setup the trainer

In [11]:
# print(all_data)
trainer = Trainer(model=model,
                args=training_args,
                data_collator=data_collator,
                train_dataset=df_train.map(process_example, remove_columns=list(df_train[0].keys())),
                eval_dataset=df_dev.map(process_example, remove_columns=list(df_dev[0].keys())),
                compute_metrics=compute_metrics,
                )

Map:   0%|          | 0/41600 [00:00<?, ? examples/s]

Map:   0%|          | 0/5200 [00:00<?, ? examples/s]

Using auto half precision backend


<h1> Train

In [12]:
training = trainer.train()

***** Running training *****
  Num examples = 41,600
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 6,500
  Number of trainable parameters = 120,785,690


Step,Training Loss,Validation Loss,F1 Score,Recall,Precision
100,3.258200,3.247994,0.041043,0.061731,0.070819
200,3.185600,3.013060,0.099585,0.131538,0.159810
300,2.859800,2.510997,0.212664,0.255769,0.243656
400,2.541100,2.280811,0.267535,0.299423,0.292639
500,2.359100,2.096139,0.304274,0.333269,0.341267
600,2.220900,2.015607,0.337418,0.358654,0.367359
700,2.056800,1.898062,0.366933,0.386923,0.380928
800,1.963900,1.827565,0.392499,0.408462,0.406971
900,1.881700,1.795691,0.397490,0.416154,0.430091
1000,1.842000,1.711817,0.414849,0.432500,0.434427


***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** Running Evaluation *****
  Num examples = 5200
  Batch size = 64
***** 

<h1> Prediction

In [13]:
pred = trainer.predict(df_test.map(process_exampleTemp, remove_columns=list(df_test[0].keys())))
df_testOrignal[prediction_label] = [categories[x] for x in np.argmax(pred.predictions[0], axis=1)]

Parameter 'function'=<function process_exampleTemp at 0x7fac08b55750> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/5200 [00:00<?, ? examples/s]

***** Running Prediction *****
  Num examples = 5200
  Batch size = 64


<h1> Saving

In [14]:
df_testOrignal[[prediction_label]].to_csv(file_save+'/madar.tsv', index=False, header=False, sep="\t")
pd.read_csv(file_save+"/madar.tsv", sep="\t").head(3)

,MSA
0,ALE
1,SAN
2,AMM


In [15]:
print("done")

done
